# Get API Datas

In [ ]:
from googleapiclient.discovery import build
from google.oauth2 import service_account
from datetime import datetime, timedelta

# Set your service account information
SERVICE_ACCOUNT_FILE = 'example.json'
SCOPES = ['https://www.googleapis.com/auth/webmasters.readonly']

# Set your site URL
SITE_URL = "https://example.com/"

# Set the start and end dates
start_date = datetime(2024, 2, 1)
end_date = datetime(2024, 8, 7)

# Maximum number of rows per API request
maxRows = 25000

# Create a list to store the output data
output_rows = []

def main():
    credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

    service = build('searchconsole', 'v1', credentials=credentials)

    # Iterate through the date range
    for date in date_range(start_date, end_date):
        date_str = date.strftime("%Y-%m-%d")
        print(f"Fetching data for {date_str}...")

        i = 0
        while True:
            request = {
                'startDate': date_str,
                'endDate': date_str,
                'dimensions': ["query", "page", "country", "device"],
                "searchType": "Web",
                'rowLimit': maxRows,
                'startRow': i * maxRows
            }

            response = service.searchanalytics().query(siteUrl=SITE_URL, body=request).execute()

            if 'rows' not in response:
                print(f"No data received for {date_str}. Continuing...")
                break

            for row in response['rows']:
                keyword = row['keys'][0]
                page = row['keys'][1]
                country = row['keys'][2]
                device = row['keys'][3]
                output_row = [date_str, keyword, page, country, device, row['clicks'], row['impressions'],
                              row['ctr'],
                              row['position']]
                output_rows.append(output_row)

            if len(response['rows']) < maxRows:
                # Reached the last page, move on to the next day
                break
            else:
                i += 1

    # Process the data (e.g., write to a CSV file)
    process_data(output_rows)

def date_range(start, end):
    for n in range(int((end - start).days) + 1):
        yield start + timedelta(n)

def process_data(data):
    import csv
    with open('search_console_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(
            ["Date", "Keyword", "Page", "Country", "Device", "Clicks", "Impressions", "CTR", "Position"])
        writer.writerows(data)  # Write the raw data directly


if __name__ == "__main__":
    main()

# average_gun Calculate
#### When canonicalizing URLs, it provides additional data to help decide which URL to canonicalize. (The ratio of the number of days the URL received impressions to the total number of days from the date of the first impression to the current date: average_gun)

In [ ]:
import pandas as pd
from datetime import datetime

# Load the data set
df = pd.read_csv("search_console_data.csv")

# Convert the Date column to datetime object
df['Date'] = pd.to_datetime(df['Date'])

# Empty lists for the results DataFrame
results = []

# Perform calculations for each URL
for url in df['Page'].unique():
    # Filter data for the URL
    url_data = df[df['Page'] == url].copy()

    # Find the first impression date
    ilk_gosterim_tarihi = url_data[url_data['Position'] > 0]['Date'].min()

    # Calculate the number of days passed since the first impression date until 06.08.2024
    toplam_gun = (datetime(2024, 8, 7) - ilk_gosterim_tarihi).days + 1             #CHANGE THIS WITH CURRENT DATE 👈👈👈

    # Filter data from the first impression date to 06.08.2024
    url_data = url_data[url_data['Date'] >= ilk_gosterim_tarihi]

    # Find the days with impressions (use unique() to count only once)
    gosterim_gunleri = url_data[url_data['Position'] > 0]['Date'].unique()
    gosterim_gun_sayisi = len(gosterim_gunleri)

    # Calculate the average impression day
    average_gun = gosterim_gun_sayisi / toplam_gun

    # Calculate the average of Position values (only those greater than 0)
    ortalama_position = url_data[url_data['Position'] > 0]['Position'].mean()

    # Calculate the square of differences from the average for each day
    kare_farklar = [(p - ortalama_position)**2 for p in url_data[url_data['Position'] > 0]['Position']]

    # Calculate the square root of the average of squared differences (standard deviation)
    standart_sapma = (sum(kare_farklar) / len(kare_farklar))**0.5

    # Calculate the coefficient of variation
    degisim_kaysayısı = (standart_sapma / ortalama_position) * 100 if ortalama_position != 0 else 0

    # Add the results to the list
    results.append([url, average_gun, gosterim_gun_sayisi, toplam_gun, degisim_kaysayısı])

# Create the results DataFrame
results_df = pd.DataFrame(results, columns=['URL', 'average_gun', 'gosterim_gun_sayisi', 'toplam_gun', 'degisim_kaysayısı'])

results_df['average_gun'] = results_df['average_gun'].round(2)

# Save the results to a CSV file
results_df.to_csv("sonuclarFINAL2.csv", index=False)

# Grouping 

In [ ]:
import pandas as pd

def group_and_aggregate_data(csv_file_path):
    """
    Groups data by URL, combines same keywords for the same URL,
    and sums the click and impression values.

    Args:
        csv_file_path (str): Path to the CSV file to process.

    Returns:
        pd.DataFrame: Grouped and aggregated data.
    """

    df = pd.read_csv(csv_file_path, encoding='utf-8')
    
    df['Include'] = df.groupby(['Page', 'Keyword'])['Clicks'].transform('sum') > 0
    
    df_filtered = df[df['Include']].copy()

    grouped_data = df_filtered.groupby('Page').agg(
        Keywords=('Keyword', lambda x: '|'.join(set(x))),  # Combine unique keywords with '|'
        TotalClicks=('Clicks', 'sum'),
        TotalImpressions=('Impressions', 'sum')
    ).reset_index()

    return grouped_data

def write_grouped_data_to_csv(grouped_data, output_file_path):
    """
    Writes the grouped and aggregated data to a CSV file.

    Args:
        grouped_data (pd.DataFrame): Grouped data.
        output_file_path (str): Path to the output CSV file.
    """

    grouped_data.to_csv(output_file_path, index=False, encoding='utf-8')

if __name__ == "__main__":
    csv_file_path = 'search_console_data.csv'  # Path to the input CSV file
    output_file_path = 'grouped_search_console_data.csv'  # Path to the output CSV file

    grouped_data = group_and_aggregate_data(csv_file_path)
    write_grouped_data_to_csv(grouped_data, output_file_path)
    print(f"Grouped data written to '{output_file_path}' file.")

# Calculating similarity between clusters

In [ ]:
import pandas as pd
import re
from fuzzywuzzy import fuzz
from tqdm import tqdm
from collections import Counter

df = pd.read_csv("grouped_search_console_data.csv")
search_console_df = pd.read_csv("search_console_data.csv")

def create_keyword_sets(row):
    keywords = row['Keywords'].split('|')
    excluded_pattern = r"YOUR REGEX"    # Excludes words specified with regex
    keywords = [word for word in keywords if not re.search(excluded_pattern, word)]
    return set(keywords)

df['Keyword Sets'] = df.apply(create_keyword_sets, axis=1)
pages = df['Page'].tolist()

similarity_results = []
total_iterations = (len(pages) * (len(pages) - 1)) // 2

with tqdm(total=total_iterations, desc="Smilarity Calculating") as pbar:
    for i in range(len(pages)):
        for j in range(i + 1, len(pages)):
            page1_keywords = df['Keyword Sets'][i]
            page2_keywords = df['Keyword Sets'][j]

            str_page1_keywords = ' '.join(page1_keywords)
            str_page2_keywords = ' '.join(page2_keywords)

            similarity_sort = fuzz.token_sort_ratio(str_page1_keywords, str_page2_keywords)
            similarity_set = fuzz.token_set_ratio(str_page1_keywords, str_page2_keywords)

            #fuzzywuzzy smilarity
            if similarity_set - similarity_sort >= 60:
                similarity = (similarity_set * 0.6) + (similarity_sort * 0.4)
            else:
                similarity = (similarity_sort * 0.6) + (similarity_set * 0.4)

            #jaccard smilarity
            intersection = len(page1_keywords.intersection(page2_keywords))
            union = len(page1_keywords.union(page2_keywords))
            jaccard_similarity = intersection / union if union > 0 else 0
            combined_score = (0.7 * similarity) + (0.3 * jaccard_similarity * 100) #combined smilarity

            if jaccard_similarity > 0:
                similarity_results.append({
                    'Page 1': pages[i],
                    'Page 2': pages[j],
                    'Similarity (%)': int(similarity),
                    'Jaccard Similarity (%)': int(jaccard_similarity * 100),
                    'Combined Score (%)': int(combined_score)
                })

            pbar.update(1)

similarity_df = pd.DataFrame(similarity_results)
similarity_df = similarity_df[similarity_df['Similarity (%)'] >= 60]


def find_top_keywords(page1, page2):

    page1_keywords = search_console_df[search_console_df['Page'] == page1].groupby('Keyword').agg({'Clicks': 'sum', 'Impressions': 'sum'})
    page2_keywords = search_console_df[search_console_df['Page'] == page2].groupby('Keyword').agg({'Clicks': 'sum', 'Impressions': 'sum'})


    common_keywords = list(set(page1_keywords.index) & set(page2_keywords.index))

    if common_keywords:
 
        keyword_data = pd.concat([page1_keywords.loc[common_keywords], page2_keywords.loc[common_keywords]])
        

        filtered_keyword_data = keyword_data[(keyword_data['Clicks'] > 0).groupby(keyword_data.index).transform('all')]
        

        keyword_counts = filtered_keyword_data.groupby('Keyword').agg({'Clicks': 'sum', 'Impressions': 'sum'})


        top_clicks_keyword = keyword_counts['Clicks'].idxmax() if not keyword_counts.empty else None
        top_clicks_count = keyword_counts['Clicks'].max() if not keyword_counts.empty else None


        top_impressions_keyword = keyword_counts['Impressions'].idxmax() if not keyword_counts.empty else None
        top_impressions_count = keyword_counts['Impressions'].max() if not keyword_counts.empty else None

        return top_clicks_keyword, top_clicks_count, top_impressions_keyword, top_impressions_count
    else:
        return None, None, None, None


similarity_df['Top Clicks Keyword'] = None
similarity_df['Top Clicks Count'] = None
similarity_df['Top Impressions Keyword'] = None
similarity_df['Top Impressions Count'] = None

for index, row in similarity_df.iterrows():
    top_clicks_keyword, top_clicks_count, top_impressions_keyword, top_impressions_count = \
        find_top_keywords(row['Page 1'], row['Page 2'])

    similarity_df.loc[index, 'Top Clicks Keyword'] = top_clicks_keyword
    similarity_df.loc[index, 'Top Clicks Count'] = top_clicks_count
    similarity_df.loc[index, 'Top Impressions Keyword'] = top_impressions_keyword
    similarity_df.loc[index, 'Top Impressions Count'] = top_impressions_count

print(similarity_df)
similarity_df.to_csv("final.csv", index=False)